In [53]:
# Import the 
import pandas as pd 
import numpy as np 
import os
import time 

os.chdir("/Users/jackychen/Penn Dropbox/Jacky Chen/JPMCI")

# Read the data
state = pd.read_excel("Data/indarte_bk_exemptions_2021_version.xlsx", sheet_name="State Exemptions")
county = pd.read_excel("Data/indarte_bk_exemptions_2021_version.xlsx", sheet_name="County Exemptions")


In [54]:
# %% DEFINE HELPERS ---------------------------------------
def fed_or_county(fed, county_val, fed_val):
    """
    This function determines, when Federal exemption is available, whether to 
    to adopt the Federal level or County level
    """
    possible_fed_val = fed * fed_val
    if possible_fed_val >= county_val:
        return possible_fed_val
    else:
        return county_val

def parse_he(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in ('unlimited', 'inf', 'infinite'):
        return np.inf
    return pd.to_numeric(s, errors='coerce')


In [55]:
# %% COMPLETE NY AND WA YEAR PANELS ---------------------------------------

# --- Setup FED table once (year -> HE_FED) ---
FED = (
    state.loc[state['state'].str.casefold() == 'federal', ['year', 'HE']]
         .drop_duplicates(subset=['year'])
         .rename(columns={'HE': 'HE_FED'})
)

FED['HE_FED'] = pd.to_numeric(FED['HE_FED'], errors='coerce').astype('Int64')


def build_state_panel(county_df, state_fips, state_code):
    # Subset counties for the state
    st = county_df.loc[county_df['state_FIPS'] == state_fips, ['state', 'state_FIPS', 'year', 'HE', 'federal_possible']].copy()

    # Merge federal by year
    st = st.merge(FED, on='year', how='left')

    # Coerce types
    st['HE'] = pd.to_numeric(st['HE'], errors='coerce')
    st['HE_FED'] = pd.to_numeric(st['HE_FED'], errors='coerce').fillna(-np.inf)
    st['federal_possible'] = pd.to_numeric(st['federal_possible'], errors='coerce').fillna(0).astype('Int64')

    # Decide county vs federal
    st['HE'] = np.where(
        st['federal_possible'].eq(1),
        np.maximum(st['HE'], st['HE_FED']),
        st['HE']
    )

    # Median across counties for each year
    panel = (
        st.groupby('year', as_index=False)
          .agg(HE=('HE', 'median'))
          .assign(state=state_code, 
                  state_FIPS=state_fips, 
                  federal_possible = 0
                  )
          .loc[:, ['state', 'state_FIPS', 'year', 'HE', 'federal_possible']]
          .sort_values('year', kind='stable')
          .reset_index(drop=True)
    )
    return panel

# Ensure base columns are numeric once
county['state_FIPS'] = pd.to_numeric(county['state_FIPS'], errors='coerce').astype('Int64')
county['federal_possible'] = pd.to_numeric(county['federal_possible'], errors='coerce').astype('Int64')

# Create 
NY_panel = build_state_panel(county, 36, 'NY')
WA_panel = build_state_panel(county, 53, 'WA')


In [56]:
# %% COMPLETE STATE-YEAR PANELS  ---------------------------------------
state_panel = state[['state', 'state_FIPS', 'year', 'HE', 'federal_possible']].copy()

# Set state_FIPS and federal_possible to intergers
state_panel['state_FIPS'] = pd.to_numeric(state_panel['state_FIPS'], errors='coerce').astype('Int64')
state_panel['federal_possible'] = pd.to_numeric(state_panel['federal_possible'], errors='coerce').astype('Int64')

# Convert all 'Unlimited' to numeric inf 
state_panel['HE'] = state_panel['HE'].apply(parse_he)

# Add the NY and WA data to State data
state_panel = pd.concat([state_panel, NY_panel], ignore_index=True) 
state_panel = pd.concat([state_panel, WA_panel], ignore_index=True)

# Process the AR and CA data (choose the set of exemption that  is greater)
# Extract relevant data
special_states = ["AR1", "AR2", "CA1", "CA2"]
mask = state_panel['state'].isin(special_states)
special_states_panels = state_panel.loc[mask].copy()
normal_states_panels = state_panel.loc[~mask].copy()

# Procss "Unlimited" exemption level
special_states_panels["HE"] = special_states_panels["HE"].apply(parse_he)

# Collapse by base state (AR/CA) by year and take the max HE
special_states_panels["base_state"] = special_states_panels['state'].str.extract(r'^(AR|CA)')[0]


collapsed_special_states = (
    special_states_panels 
    .groupby(['base_state', 'year'], as_index=False)
    .agg(
        HE = ("HE", 'max'), 
        state_FIPS = ("state_FIPS", 'first'), 
        federal_possible = ('federal_possible', 'max')
    )
    .rename(columns = {'base_state': 'state'})
)

# Append the processed AR & CA data back
state_panel = pd.concat([normal_states_panels, collapsed_special_states], ignore_index=True)

# Process all remaining states
state_panel = pd.merge(
    state_panel, FED, on = 'year', how = 'left'
)

# Clean the NAs in federal_possible var 
state_panel['federal_possible'] = state_panel['federal_possible'].fillna(0)

# Decide state vs federal
state_panel["HE_final"] = np.where(
    state_panel['federal_possible'] == 1, 
    np.maximum(state_panel["HE"], state_panel["HE_FED"]), 
    state_panel["HE"]
)











In [57]:
# %% COMPLETE NY AND WA YEAR PANELS ---------------------------------------

# Compute yearly median exemption level 
median_exemption = (
    state_panel[state_panel['state'] != 'Federal']
    .groupby('year', as_index=False)
    .agg(
        HE_median = ("HE_final", 'median'), 
        year = ('year', 'first')
    )
    .loc[:, ["year", "HE_median"]]
)

# Merge the median exemption level back to state-year panel
state_panel_fin = pd.merge(state_panel, median_exemption, on = 'year', how = 'left')
state_panel_fin['above_median'] = np.where(
    state_panel_fin['HE_final'] > state_panel_fin['HE_median'], 
    1, 
    0
)

# Save the result 
state_panel_fin = state_panel_fin[['state', 'state_FIPS', 'year', 'above_median']]
state_panel_fin.to_csv("result/state_year_panel.csv", index=False)

In [58]:
state_panel_fin

,state,state_FIPS,year,above_median
0,AK,2,2021,0
1,AK,2,2020,0
2,AK,2,2019,0
3,AK,2,2018,0
4,AK,2,2017,0
...,...,...,...,...
1139,CA,6,2017,0
1140,CA,6,2018,0
1141,CA,6,2019,0
1142,CA,6,2020,0
